In [ ]:
!pip install rdkit
!pip install transformers==4.52.2 datasets==2.14.4 tokenizers==0.21.1

In [ ]:
# Download SA Score script (by Ertl)
!wget https://raw.githubusercontent.com/rdkit/rdkit/master/Contrib/SA_Score/sascorer.py
!wget https://github.com/rdkit/rdkit/raw/master/Contrib/SA_Score/fpscores.pkl.gz

In [ ]:
import random, os, numpy as np, pandas as pd

def seed_everything(seed=42):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

In [ ]:
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

In [ ]:
df = pd.read_csv('USPTO_50K.csv')
df

In [ ]:
# import pandas as pd
# from rdkit import Chem
# from rdkit.Chem import Descriptors
# import sascorer
# import numpy as np
# from tqdm.notebook import tqdm  # or tqdm.auto for non-Colab use

# # Enable tqdm with pandas apply
# tqdm.pandas()

# # If not already loaded:
# # df = pd.read_csv("your_uspto_50k.csv")  # make sure it has a 'reactions' column

# def compute_difficulty_metrics(reaction_smiles):
#     try:
#         product_smiles = reaction_smiles.split(">")[-1]
#         mol = Chem.MolFromSmiles(product_smiles)
#         if mol is None:
#             return np.nan, np.nan, np.nan, np.nan

#         sa_score = sascorer.calculateScore(mol)
#         ring_count = mol.GetRingInfo().NumRings()
#         heavy_atom_count = Descriptors.HeavyAtomCount(mol)

#         difficulty_score = (
#             0.5 * sa_score +
#             0.3 * ring_count +
#             0.2 * heavy_atom_count
#         )

#         return sa_score, ring_count, heavy_atom_count, difficulty_score

#     except Exception:
#         return np.nan, np.nan, np.nan, np.nan

# # Apply with progress bar
# df[['sa_score', 'ring_count', 'heavy_atoms', 'difficulty']] = df['reactions'].progress_apply(
#     compute_difficulty_metrics
# ).apply(pd.Series)

# # Remove problematic rows and sort
# df_sorted = df.dropna(subset=['difficulty']).sort_values('difficulty').reset_index(drop=True)

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(8, 5))
# sns.histplot(df_sorted['difficulty'], bins=40, kde=True)
# plt.title("Distribution of Retrosynthesis Difficulty Scores")
# plt.xlabel("Difficulty Score")
# plt.ylabel("Frequency")
# plt.grid(True)
# plt.show()

In [ ]:
from rdkit import Chem

def canonicalize_smiles(smi):
    """Canonicalizes multi-fragment SMILES (e.g. 'CC.O') with dot handling."""
    parts = smi.split('.')
    mols = [Chem.MolFromSmiles(part) for part in parts]
    if any(m is None for m in mols):
        return smi  # fallback if any fragment fails
    canonical_parts = [Chem.MolToSmiles(m, canonical=True) for m in mols]
    return '.'.join(sorted(canonical_parts))  # sort ensures consistent order

In [ ]:
# Split product/reactants from reaction SMILES
df['product'] = df['reactions'].apply(lambda x: x.split('>')[-1])
df['reactants'] = df['reactions'].apply(lambda x: x.split('>')[0])

# Canonicalize both sides
df['product'] = df['product'].apply(canonicalize_smiles)
df['reactants'] = df['reactants'].apply(canonicalize_smiles)

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)  # 81/9/10

In [ ]:
train_df = train_df.sample(frac=0.5, random_state=42).reset_index(drop=True)
# train_df = train_df.sort_values(by="difficulty").reset_index(drop=True)

In [ ]:
from transformers import AutoTokenizer, EncoderDecoderModel
from datasets import Dataset

chemberta_tokenizer = AutoTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")
gpt2_tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

# Add pad token if missing
if gpt2_tokenizer.pad_token is None:
    gpt2_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def tokenize_function(example):
    # Encoder input = product SMILES, tokenized with ChemBERTa
    input_ = chemberta_tokenizer(
        "retro: " + example['product'],
        padding="max_length",
        truncation=True,
        max_length=128
    )

    # Decoder target = reactants SMILES, tokenized with GPT2 tokenizer
    target = gpt2_tokenizer(
        example['reactants'],
        padding="max_length",
        truncation=True,
        max_length=128
    )

    input_['labels'] = target['input_ids']
    return input_

train_ds = Dataset.from_pandas(train_df[['product', 'reactants']])
val_ds = Dataset.from_pandas(val_df[['product', 'reactants']])
test_ds = Dataset.from_pandas(test_df[['product', 'reactants']])

train_tokenized = train_ds.map(tokenize_function, batched=False)
val_tokenized = val_ds.map(tokenize_function, batched=False)
test_tokenized = test_ds.map(tokenize_function, batched=False)

In [ ]:
from transformers import EncoderDecoderModel, Trainer, TrainingArguments, AutoTokenizer
import torch
import os

# Set output path
output_dir = "CurriculumLearning/retrosyn-chemberta-distilgpt2-50"

# Load tokenizers
chemberta_tokenizer = AutoTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")
gpt2_tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

# Ensure pad token exists in GPT2
if gpt2_tokenizer.pad_token is None:
    gpt2_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Look for latest checkpoint
checkpoints = sorted(
    [ckpt for ckpt in os.listdir(output_dir) if ckpt.startswith("checkpoint-")],
    key=lambda x: int(x.split("-")[-1])
)
resume_checkpoint = os.path.join(output_dir, checkpoints[-1]) if checkpoints else None

if resume_checkpoint:
    print(f"✅ Resuming weights from: {resume_checkpoint}")
    model = EncoderDecoderModel.from_encoder_decoder_pretrained(
        "seyonec/ChemBERTa-zinc-base-v1",
        "distilgpt2"
    )
    model.load_state_dict(torch.load(os.path.join(resume_checkpoint, "pytorch_model.bin")))
else:
    print("🚀 Starting from scratch")
    model = EncoderDecoderModel.from_encoder_decoder_pretrained(
        "seyonec/ChemBERTa-zinc-base-v1",
        "distilgpt2"
    )


# Set model config
model.config.decoder_start_token_id = gpt2_tokenizer.bos_token_id
model.config.pad_token_id = gpt2_tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size
model.decoder.resize_token_embeddings(len(gpt2_tokenizer))

# TrainingArguments
training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy="no",
    save_strategy="steps",
    save_steps=200,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=8,
    eval_accumulation_steps=2,
    prediction_loss_only=False,
    num_train_epochs=100,  # total epochs from scratch or resuming
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
    load_best_model_at_end=False,
    fp16=True
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=gpt2_tokenizer
)

# 🚀 Start (or resume) training
trainer.train()

In [ ]:
trainer.save_model(output_dir)